# Automated Machine Learning

In [26]:

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
 

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [27]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

AZML
RG-AZML
eastus
a748a023-896b-4719-8f8e-3d1ba62d6e35


In [28]:
# Choose a name for the experiment.
experiment_name = 'ideb-regression'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Subscription ID,a748a023-896b-4719-8f8e-3d1ba62d6e35
Workspace,AZML
Resource Group,RG-AZML
Location,eastus
Run History Name,ideb-regression


## Using AmlCompute

In [29]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

### Load Data

In [30]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'a748a023-896b-4719-8f8e-3d1ba62d6e35'
resource_group = 'RG-AZML'
workspace_name = 'AZML'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='dataset2019ideb')
dataset.to_pandas_dataframe()

,Sigla_UF,Cod_Municipio,Nome_Municipio,Cod_Escola,Nome_Escola,Rede,TAprov2019_Total,TAprov2019_1_serie,TAprov2019_2_serie,TAprov2019_3_serie,TAprov2019_4_serie,Indicador_Rendimento,SAEB2019_Matematica,SAEB2019_Lingua Portuguesa,SAEB2019_Nota Media Padronizada,IDEB_2019
0,BA,2914703,Itaberaba,29083524,EE - COLEGIO ESTADUAL LIBERDADE,Estadual,43.3,33.7,37.3,69.8,95.0,0.42,217.25,226.70,3.13,1.3
1,AM,1300060,Amaturá,13103431,ESC EST INDIGENA MANUEL JOAQUIM SALDANHA FILHO TCHAIAREECU,Estadual,67.3,59.1,53.5,92.5,95.0,0.65,195.40,188.21,2.25,1.5
2,PA,1506708,Santana do Araguaia,15171582,EEEM JORCELI SESTARI SILVA ANEXO IV,Estadual,57.1,52.4,50.0,72.2,95.0,0.57,202.24,211.64,2.70,1.5
3,PI,2211001,Teresina,22128441,U E NOSSA SENHORA DO PERPETUO SOCORRO,Estadual,53.8,25.0,58.3,97.5,95.0,0.45,239.39,237.57,3.61,1.6
4,PI,2211001,Teresina,22131159,CETI PEDRA MOLE,Estadual,41.7,30.6,31.1,78.7,95.0,0.39,259.57,260.92,4.24,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12074,ES,3205200,Vila Velha,32078498,IFES - CAMPUS VILA VELHA,Federal,95.4,94.8,97.7,94.1,95.0,0.96,394.30,363.46,7.67,7.3
12075,PE,2611606,Recife,26119730,ESCOLA DE APLICACAO DO RECIFE,Estadual,98.2,100.0,94.4,100.0,95.0,0.98,396.83,360.43,7.66,7.5
12076,SP,3506003,Bauru,35922948,COL TEC INDUSTRIAL PROF ISAAC PORTAL ROLDAN UNESP,Estadual,98.8,97.1,100.0,99.2,95.0,0.99,393.83,356.23,7.55,7.5
12077,PE,2611606,Recife,26124297,COLEGIO DE APLICACAO DO CE DA UFPE,Federal,98.9,96.7,100.0,100.0,95.0,0.99,402.96,362.42,7.77,7.7


### Split the dataset

## Automl Settings

In [31]:
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'r2_score',
    "enable_early_stopping": True, 
    "experiment_timeout_hours": 0.5, 
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
     "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'regression',
                             compute_target = compute_target,
                             training_data = train_data,
                             label_column_name = "IDEB_2019",
                             **automl_settings
                            )



In [32]:
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster
The validation results are as follows:
Y is empty


ValidationException: ValidationException:
	Message: Validation error(s): [{
    "additional_properties": {
        "debugInfo": null
    },
    "code": "UserError",
    "severity": 2,
    "message": "Y is empty",
    "message_format": "{dataName} is empty",
    "message_parameters": {
        "dataName": "Y"
    },
    "reference_code": null,
    "details_uri": null,
    "target": "training_data",
    "details": [
        {
            "additional_properties": {
                "debugInfo": null
            },
            "code": null,
            "severity": null,
            "message": "null",
            "message_format": null,
            "message_parameters": {},
            "reference_code": null,
            "details_uri": null,
            "target": null,
            "details": [],
            "inner_error": null
        }
    ],
    "inner_error": {
        "additional_properties": {},
        "code": "BadData",
        "inner_error": {
            "additional_properties": {},

## Result

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion()

## Best Model

In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

## Test

In [ ]:
# preview the first 3 rows of the dataset

test_data = test_data.to_pandas_dataframe()
y_test = test_data["IDEB_2019"].fillna(0)
test_data = test_data.drop("IDEB_2019", 1)
test_data = test_data.fillna(0)


train_data = train_data.to_pandas_dataframe()
y_train = train_data["IDEB_2019"].fillna(0)
train_data = train_data.drop("IDEB_2019", 1)
train_data = train_data.fillna(0)

In [ ]:
y_pred_train = fitted_model.predict(train_data)
y_residual_train = y_train - y_pred_train

y_pred_test = fitted_model.predict(test_data)
y_residual_test = y_test - y_pred_test

In [ ]:
%matplotlib inline
from sklearn.metrics import mean_squared_error, r2_score

# Set up a multi-plot chart.
f, (a0, a1) = plt.subplots(1, 2, gridspec_kw = {'width_ratios':[1, 1], 'wspace':0, 'hspace': 0})
f.suptitle('Regression Residual Values', fontsize = 18)
f.set_figheight(6)
f.set_figwidth(16)

# Plot residual values of training set.
a0.axis([0, 360, -100, 100])
a0.plot(y_residual_train, 'bo', alpha = 0.5)
a0.plot([-10,360],[0,0], 'r-', lw = 3)
a0.text(16,170,'RMSE = {0:.2f}'.format(np.sqrt(mean_squared_error(y_train, y_pred_train))), fontsize = 12)
a0.text(16,140,'R2 score = {0:.2f}'.format(r2_score(y_train, y_pred_train)),fontsize = 12)
a0.set_xlabel('Training samples', fontsize = 12)
a0.set_ylabel('Residual Values', fontsize = 12)

# Plot residual values of test set.
a1.axis([0, 90, -100, 100])
a1.plot(y_residual_test, 'bo', alpha = 0.5)
a1.plot([-10,360],[0,0], 'r-', lw = 3)
a1.text(5,170,'RMSE = {0:.2f}'.format(np.sqrt(mean_squared_error(y_test, y_pred_test))), fontsize = 12)
a1.text(5,140,'R2 score = {0:.2f}'.format(r2_score(y_test, y_pred_test)),fontsize = 12)
a1.set_xlabel('Test samples', fontsize = 12)
a1.set_yticklabels([])

plt.show()

In [ ]:
%matplotlib inline
test_pred = plt.scatter(y_test, y_pred_test, color='')
test_test = plt.scatter(y_test, y_test, color='g')
plt.legend((test_pred, test_test), ('prediction', 'truth'), loc='upper left', fontsize=8)
plt.show()